# Finding Available Rates

We can get a single library with all the rates known to pynucastro---this is a combination of {py:obj}`ReacLibLibrary <pynucastro.rates.library.ReacLibLibrary>`, all of the different collections that can be part of {py:obj}`TabularLibrary <pynucastro.rates.library.TabularLibrary>`, and the rates contained in {py:mod}`alternate_rates <pynucastro.rates.alternate_rates>`.  This is done via {py:meth}`full_library <pynucastro.rates.library.full_library>`.

```{note}
The {py:obj}`Library <pynucastro.rates.library.Library>` returned by `full_library` will have duplicates for some links, since multiple sources can provide a rate.
```

In [1]:
import pynucastro as pyna

In [2]:
lib = pyna.full_library()

We can see the total number of rates

In [3]:
lib.num_rates

82142

We can then use a {py:obj}`RateFilter <pynucastro.rates.library.RateFilter>` to find rates.

For example, to find all the rates that involve ${}^{12}\mathrm{C}$ as a reactant, we can do:

In [4]:
rf = pyna.RateFilter(reactants=["c12"], exact=False)
rates = lib.filter(rf).get_rates()

We can look at the source from the literature and the pynucastro class for each of the rates:

In [5]:
for r in sorted(rates):
    print(f"{str(r):24} {r.source['Label']:6}  {type(r).__name__}")

C12 + n ⟶ He4 + Be9      cf88    ReacLibRate
C12 + n ⟶ p + B12        wag     ReacLibRate
C12 + n ⟶ C13 + 𝛾        ks03    ReacLibRate
C12 + p ⟶ N13 + 𝛾        ls09    ReacLibRate
C12 + He4 ⟶ p + N15      nacr    ReacLibRate
C12 + He4 ⟶ n + O15      cf88    ReacLibRate
C12 + He4 ⟶ O16 + 𝛾      debo    DeBoerC12agO16
C12 + He4 ⟶ O16 + 𝛾      nac2    ReacLibRate
C12 + C12 ⟶ He4 + Ne20   cf88    ReacLibRate
C12 + C12 ⟶ p + Na23     cf88    ReacLibRate
C12 + C12 ⟶ n + Mg23     cf88    ReacLibRate
C12 ⟶ 3 He4              fy05    ReacLibRate
C12 ⟶ p + B11            nw00    ReacLibRate
C12 ⟶ n + C11            bb92    ReacLibRate
O16 + C12 ⟶ He4 + Mg24   cf88    ReacLibRate
O16 + C12 ⟶ p + Al27     cf88    ReacLibRate
O16 + C12 ⟶ n + Si27     cf88    ReacLibRate
Ne20 + C12 ⟶ He4 + Si28  rolf    ReacLibRate
Ne20 + C12 ⟶ p + P31     rolf    ReacLibRate
Ne20 + C12 ⟶ n + S31     rolf    ReacLibRate


Here we see that there are 2 sources for ${}^{12}\mathrm{C}(\alpha,\gamma){}^{16}\mathrm{O}$: NACRE and DeBoer et al.